# 1. std::unique_ptr` in C++

In C++, `std::unique_ptr` è un tipo di smart pointer fornito dalla Standard Library che gestisce automaticamente la durata di un oggetto. È particolarmente utile per evitare problemi di gestione della memoria, come i memory leaks, che sono comuni quando si usano i puntatori classici.

### Vantaggi di `std::unique_ptr`

1. **Gestione automatica della memoria**: `std::unique_ptr` garantisce che l'oggetto puntato venga deallocato automaticamente quando il `unique_ptr` esce dallo scope, eliminando la necessità di chiamare esplicitamente `delete`.
2. **No Memory Leaks**: Poiché la memoria viene gestita automaticamente, si riduce drasticamente il rischio di memory leaks.
3. **Unicità**: Come suggerisce il nome, un `std::unique_ptr` possiede in modo univoco l'oggetto puntato. Non ci possono essere due `std::unique_ptr` che puntano allo stesso oggetto, il che previene accessi concorrenti non sicuri.
4. **Movimentazione**: `std::unique_ptr` supporta il movimento (move semantics), consentendo di trasferire la proprietà dell'oggetto senza costi di copia.

### Utilizzo di `std::unique_ptr`

Ecco una panoramica di come usare `std::unique_ptr`:

#### Creazione di un `std::unique_ptr`

```cpp
#include <memory>
#include <iostream>

class Resource {
public:
    Resource() { std::cout << "Resource acquired\n"; }
    ~Resource() { std::cout << "Resource destroyed\n"; }
    void doSomething() { std::cout << "Using resource\n"; }
};

int main() {
    // Creazione di un std::unique_ptr che gestisce un'istanza di Resource
    std::unique_ptr<Resource> ptr(new Resource());
    ptr->doSomething();

    // Il Resource viene automaticamente deallocato quando ptr esce dallo scope
    return 0;
}
```

#### Utilizzo di `std::make_unique`

È preferibile usare `std::make_unique` per creare un `std::unique_ptr` perché è più sicuro e meno soggetto a errori:

```cpp
#include <memory>
#include <iostream>

int main() {
    // Uso di std::make_unique per creare il std::unique_ptr
    std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
    ptr->doSomething();

    return 0;
}
```

#### Movimentazione di un `std::unique_ptr`

Poiché `std::unique_ptr` non può essere copiato, può essere spostato usando `std::move`:

```cpp
#include <memory>
#include <iostream>

void processResource(std::unique_ptr<Resource> res) {
    res->doSomething();
    // Il Resource verrà deallocato quando res esce dallo scope
}

int main() {
    std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
    processResource(std::move(ptr));

    // A questo punto, ptr non possiede più il Resource
    if (!ptr) {
        std::cout << "ptr is now nullptr\n";
    }

    return 0;
}
```

### Confronto con i Puntatori Classici

#### Puntatori Classici

Usare i puntatori classici richiede una gestione manuale della memoria:

```cpp
int main() {
    Resource* ptr = new Resource();
    ptr->doSomething();
    delete ptr;  // Necessario per evitare memory leaks

    return 0;
}
```

Problemi comuni:
- **Memory Leaks**: Se dimentichi di chiamare `delete`, la memoria non viene liberata.
- **Doppia Deallocazione**: Se chiami `delete` due volte sullo stesso puntatore, causerai un comportamento indefinito.
- **Eccezioni**: Se un'eccezione viene lanciata prima di `delete`, il codice potrebbe non raggiungere mai la chiamata a `delete`, causando un memory leak.

#### `std::unique_ptr`

`std::unique_ptr` risolve questi problemi gestendo automaticamente la durata dell'oggetto:

```cpp
int main() {
    {
        std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
        ptr->doSomething();
    }
    // Il Resource viene automaticamente deallocato qui

    return 0;
}
```

### Riassunto

- `std::unique_ptr` fornisce gestione automatica della memoria, prevenendo memory leaks.
- È facile da usare con `std::make_unique`, che semplifica la creazione del puntatore.
- `std::unique_ptr` è sicuro, evitando problemi di doppia deallocazione e gestendo correttamente le eccezioni.
- Usa il movimento per trasferire la proprietà senza costi di copia.

In conclusione, `std::unique_ptr` è uno strumento potente e sicuro per la gestione della memoria in C++, rendendo il codice più robusto e meno soggetto a errori rispetto all'uso dei puntatori classici.

__________________________________________________________
# 2.`std::make_unique` e `std::move` in C++

```cpp
std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
processResource(std::move(ptr));
```

### 1. Creazione di un `std::unique_ptr`

```cpp
std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
```

Questa riga di codice fa due cose:
- **Creazione dell'oggetto**: `std::make_unique<Resource>()` crea un nuovo oggetto di tipo `Resource` nel heap e restituisce un `std::unique_ptr` che lo possiede.
- **Assegnazione del puntatore**: Il `std::unique_ptr` restituito da `std::make_unique` viene assegnato alla variabile `ptr`.

### 2. Passaggio di `std::unique_ptr` a una funzione

```cpp
processResource(std::move(ptr));
```

Qui succedono diverse cose:
- **Movimentazione del `std::unique_ptr`**: `std::move(ptr)` converte `ptr` in un rvalue reference, che permette il trasferimento della proprietà del puntatore. `std::move` è una funzione che effettua un cast a `T&&` (rvalue reference) e consente la semantica di movimento.
- **Passaggio alla funzione**: Il risultato di `std::move(ptr)` viene passato alla funzione `processResource`. La firma della funzione `processResource` è simile a questa:

```cpp
void processResource(std::unique_ptr<Resource> res) {
    res->doSomething();
    // Il Resource verrà deallocato quando res esce dallo scope
}
```

#### Cosa accade esattamente durante il passaggio a `processResource`:

- **Trasferimento della proprietà**: Il `std::unique_ptr` originale (`ptr`) trasferisce la proprietà dell'oggetto puntato a `res` nella funzione `processResource`. Dopo il `std::move`, `ptr` non possiede più l'oggetto, e `ptr` diventa `nullptr`.
- **Utilizzo nella funzione**: All'interno della funzione `processResource`, `res` è ora il proprietario del `Resource` e può usarlo come desidera. Quando `res` esce dallo scope (alla fine della funzione), l'oggetto `Resource` viene automaticamente deallocato, evitando qualsiasi memory leak.

### Dettagli del processo di movimento

- **Prima di `std::move`**: `ptr` è un `std::unique_ptr<Resource>` che possiede un oggetto `Resource`.
- **Dopo `std::move`**: `ptr` è stato convertito in un rvalue reference, che permette il trasferimento della proprietà. La variabile `ptr` stessa non possiede più l'oggetto (diventa `nullptr`), e la proprietà è stata trasferita a `res`.

### Esempio Completo

Ecco un esempio completo con tutti i passaggi per chiarire:

```cpp
#include <iostream>
#include <memory>

class Resource {
public:
    Resource() { std::cout << "Resource acquired\n"; }
    ~Resource() { std::cout << "Resource destroyed\n"; }
    void doSomething() { std::cout << "Using resource\n"; }
};

void processResource(std::unique_ptr<Resource> res) {
    res->doSomething();
    // Il Resource verrà deallocato quando res esce dallo scope
}

int main() {
    // Creazione di un std::unique_ptr che gestisce un'istanza di Resource
    std::unique_ptr<Resource> ptr = std::make_unique<Resource>();
    // Trasferimento della proprietà del ptr a processResource
    processResource(std::move(ptr));

    // A questo punto, ptr non possiede più il Resource
    if (!ptr) {
        std::cout << "ptr is now nullptr\n";
    }

    return 0;
}
```

### Riassunto

- **`std::make_unique<Resource>()`**: Crea un nuovo `Resource` e restituisce un `std::unique_ptr` che lo possiede.
- **`std::move(ptr)`**: Converte `ptr` in un rvalue reference per permettere il trasferimento della proprietà.
- **`processResource(std::move(ptr))`**: Trasferisce la proprietà dell'oggetto `Resource` da `ptr` a `res` nella funzione `processResource`.

Questo uso di `std::unique_ptr` garantisce che l'oggetto `Resource` sia sempre correttamente gestito e deallocato, prevenendo memory leaks e rendendo il codice più sicuro e facile da mantenere.

# 3. std::pair vs std::tuple in C++

Le classi `std::pair` e `std::tuple` in C++ sono entrambe utilizzate per raggruppare diversi valori insieme, ma presentano alcune differenze significative in termini di funzionalità e utilizzo.

### `std::pair`

`std::pair` è una classe template che consente di memorizzare due valori, potenzialmente di tipi diversi. È definita in `<utility>`.

#### Caratteristiche di `std::pair`:

1. **Due elementi**: Contiene esattamente due membri.
2. **Accesso ai membri**: Gli elementi sono accessibili come `first` e `second`.
3. **Tipo fisso**: `std::pair` è progettato specificamente per contenere due elementi.
4. **Semplicità**: È semplice e facile da usare quando hai bisogno di una struttura per memorizzare due valori.
5. **Costruttori e Assegnazioni**: Fornisce costruttori e assegnazioni standard per inizializzare e copiare i valori.

#### Esempio di `std::pair`:

```cpp
#include <utility>
#include <iostream>

int main() {
    std::pair<int, std::string> myPair(1, "example");

    std::cout << "First: " << myPair.first << ", Second: " << myPair.second << std::endl;

    return 0;
}
```

### `std::tuple`

`std::tuple` è una classe template che consente di memorizzare un numero arbitrario di valori, potenzialmente di tipi diversi. È definita in `<tuple>`.

#### Caratteristiche di `std::tuple`:

1. **Elementi multipli**: Può contenere un numero arbitrario di membri.
2. **Accesso ai membri**: Gli elementi sono accessibili utilizzando la funzione `std::get<N>(tuple)`, dove `N` è l'indice dell'elemento.
3. **Flessibilità**: `std::tuple` è progettato per essere flessibile e può contenere più di due elementi di tipi diversi.
4. **Utility**: Fornisce funzioni aggiuntive come `std::tie` e `std::make_tuple` per una migliore manipolazione.
5. **Complessità**: È più complesso rispetto a `std::pair` e può essere meno leggibile quando contiene molti elementi.

#### Esempio di `std::tuple`:

```cpp
#include <tuple>
#include <iostream>

int main() {
    std::tuple<int, std::string, double> myTuple(1, "example", 3.14);

    std::cout << "First: " << std::get<0>(myTuple) 
              << ", Second: " << std::get<1>(myTuple) 
              << ", Third: " << std::get<2>(myTuple) << std::endl;

    return 0;
}
```

### Differenze Chiave

1. **Numero di elementi**:
   - `std::pair` contiene esattamente due elementi.
   - `std::tuple` può contenere un numero arbitrario di elementi.

2. **Accesso agli elementi**:
   - `std::pair` utilizza `first` e `second` per accedere ai suoi elementi.
   - `std::tuple` utilizza `std::get<N>` per accedere ai suoi elementi, dove `N` è l'indice.

3. **Utilità e Funzionalità**:
   - `std::pair` è più semplice e specifico per due valori.
   - `std::tuple` offre maggiore flessibilità e funzionalità, come la possibilità di utilizzare `std::tie` per decomporre un `tuple`.

4. **Performance**:
   - `std::pair` potrebbe essere leggermente più efficiente in termini di prestazioni e utilizzo della memoria per il caso specifico di due valori.
   - `std::tuple` è più generico e potrebbe introdurre un sovraccarico leggermente maggiore per l'accesso ai valori.

### Quando Usare `std::pair` vs `std::tuple`

- **Usa `std::pair`** quando hai bisogno di una struttura semplice per memorizzare due valori e vuoi un accesso rapido e leggibile con `first` e `second`.
- **Usa `std::tuple`** quando hai bisogno di memorizzare più di due valori o quando vuoi sfruttare la flessibilità di memorizzare un numero variabile di elementi di tipi diversi.

In sintesi, la scelta tra `std::pair` e `std::tuple` dipende dal contesto e dalle esigenze specifiche del tuo codice. `std::pair` è ideale per semplici coppie di valori, mentre `std::tuple` è adatto per situazioni più complesse che richiedono la gestione di più valori.

Quando si tratta di creare una tabella di codici di Huffman che memorizza delle triplette (sym, len, code), ci sono due opzioni principali: utilizzare una `std::unordered_map` con `uint64_t` come chiave e una `std::pair` come valore, oppure una `std::tuple`.

Vediamo entrambe le opzioni in dettaglio:

### Opzione 1: `std::unordered_map` con `std::pair`

In questa opzione, la chiave sarà un `uint64_t` (presumibilmente il simbolo) e il valore sarà una `std::pair` contenente la lunghezza del codice e il codice stesso.

```cpp
#include <unordered_map>
#include <utility>
#include <iostream>

int main() {
    // Creazione della tabella di codici usando unordered_map e pair
    std::unordered_map<uint64_t, std::pair<int, uint64_t>> codeTable;

    // Inserimento di una tripla (symbol, length, code)
    codeTable[1] = std::make_pair(3, 0b101);

    // Accesso ai valori
    uint64_t sym = 1;
    if (codeTable.find(sym) != codeTable.end()) {
        int len = codeTable[sym].first;
        uint64_t code = codeTable[sym].second;
        std::cout << "Symbol: " << sym << ", Length: " << len << ", Code: " << code << std::endl;
    }

    return 0;
}
```

### Opzione 2: `std::unordered_map` con `std::tuple` - you are gay ester egg

In questa opzione, la chiave sarà ancora un `uint64_t` (il simbolo) e il valore sarà una `std::tuple` contenente la lunghezza del codice e il codice stesso.

```cpp
#include <unordered_map>
#include <tuple>
#include <iostream>

int main() {
    // Creazione della tabella di codici usando unordered_map e tuple
    std::unordered_map<uint64_t, std::tuple<int, uint64_t>> codeTable;

    // Inserimento di una tripla (symbol, length, code)
    codeTable[1] = std::make_tuple(3, 0b101);

    // Accesso ai valori
    uint64_t sym = 1;
    if (codeTable.find(sym) != codeTable.end()) {
        int len = std::get<0>(codeTable[sym]);
        uint64_t code = std::get<1>(codeTable[sym]);
        std::cout << "Symbol: " << sym << ", Length: " << len << ", Code: " << code << std::endl;
    }

    return 0;
}
```

### Confronto tra `std::pair` e `std::tuple`

#### `std::pair`
- **Semplicità**: Se hai solo due elementi (length e code), `std::pair` è semplice e chiaro.
- **Accesso ai membri**: `first` e `second` sono facili da capire e da usare.
- **Leggibilità**: È immediatamente chiaro che ci sono due valori associati.

#### `std::tuple`
- **Flessibilità**: Se pensi che potresti voler aggiungere più elementi in futuro, `std::tuple` è più flessibile.
- **Accesso ai membri**: Usa `std::get<N>` per accedere agli elementi, che può essere meno leggibile ma è potente.
- **Uso esplicito dei tipi**: Può contenere un numero arbitrario di elementi di tipi diversi.

### Quale scegliere?

- **Usa `std::pair`** se hai solo due valori (length e code) e vuoi una soluzione semplice e leggibile.
- **Usa `std::tuple`** se potresti espandere la tua struttura per includere più valori in futuro o se hai già più di due valori.

In sintesi, entrambe le opzioni sono valide, ma `std::pair` è spesso preferibile per la semplicità quando si gestiscono solo due valori. Se prevedi che la tua struttura possa evolvere per includere più valori, allora `std::tuple` offre una maggiore flessibilità.